# Cardinality Constrained Portfolio Optimization

This notebook explores the **Cardinality-Constrained Mean-Variance (CCMV)** portfolio optimization problem, based on the methodology presented in *"Cardinality-Constrained Portfolio Optimization with Clustering"* (Ebrahimi et al., 2025).

The classical **Markowitz mean-variance model** is extended by adding a **cardinality constraint**, limiting the number of assets included in the portfolio. This transforms the problem into a **Mixed-Integer Quadratic Program (MIQP)**, making it significantly more challenging to solve due to its NP-hard nature.

In this notebook, we will:

- Formulate the classical and cardinality-constrained optimization problems
- Discuss the implications of binary variables and computational complexity
- Implement and solve small CCMV problems using Python
- Prepare for future integration of clustering methods as described in the paper

This work serves as both a theoretical deep dive and a practical foundation for reproducing the paper’s results.

### Portfolio Optimization Problem Formulation

The classical **mean-variance portfolio optimization** problem, introduced by Markowitz (1952), aims to minimize portfolio risk for a given level of expected return (or equivalently, balance both using a risk aversion parameter). It is formulated as:

$$
\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1,\quad 0 \leq w_i \leq 1 \quad \forall i \in \{1, \ldots, N\}
$$

where:

- $ w = (w_1, \ldots, w_N)^\top $ is the vector of portfolio weights  
- $ \Sigma \in \mathbb{R}^{N \times N} $ is the **covariance matrix** of asset returns  
- $ \mu \in \mathbb{R}^N $ is the **expected return vector**  
- $ \gamma > 0 $ is the **risk aversion coefficient**  
- $ \mathbf{1} \in \mathbb{R}^N $ is a vector of ones  
- The constraint $ \mathbf{1}^\top w = 1 $ ensures the portfolio is fully invested  
- The box constraint $ 0 \leq w_i \leq 1 $ limits individual asset weights

This is a **convex quadratic program** and can be efficiently solved using standard optimization methods.



This classical model is easy to solve due to its convexity, but it does not capture important practical constraints that investors often face. In real-world applications, it is common to:

- Reduce transaction costs  
- Control portfolio complexity  
- Avoid excessive concentration in a few assets

To reflect these needs, we introduce two key extensions:

#### 1. Cardinality Constraint

Investors may want to limit the total number of assets in the portfolio. Let $\Delta \in \mathbb{N}_+$ denote the maximum allowed number of assets. This leads to the **cardinality constraint**:

$$
\|w\|_0 \leq \Delta
$$

Here, $\|w\|_0$ is the **zero-norm**, which counts the number of nonzero entries in $w$. This constraint is **non-convex** and introduces **combinatorial complexity**, requiring the use of binary decision variables.

#### 2. Box Constraint on Asset Weights

To prevent overexposure to individual assets, we constrain each weight to lie within a maximum threshold. Let $\bar{w} \in (0, 1]$ be the maximum allowable weight per asset. The constraint is written as:

$$
w \in [0, \bar{w}]^N
$$

This **box constraint** ensures no single asset can dominate the portfolio, and also helps with regulatory compliance or risk diversification.

Together, these constraints transform the problem into a **Mixed-Integer Quadratic Program (MIQP)**, which is computationally much harder to solve than the unconstrained case.

We can then formulate the **Cardinality-Constrained Mean-Variance (CCMV)** optimization problem as follows:

$$
w^*(\Delta, \bar{w}) = \arg\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1, \quad 0 \leq w_i \leq \bar{w} \quad \forall i, \quad \|w\|_0 \leq \Delta
$$

Furthermore, it can be shown that this problem is NP-hard due to the combinatorial nature of the cardinality constraint. The introduction of binary variables to enforce this constraint complicates the optimization significantly.


### Solving the CCMV Problem

The **Cardinality-Constrained Mean-Variance (CCMV)** problem is computationally challenging due to its non-convex nature and the presence of binary decisions. To address this, we adopt the **clustering-based approach** proposed in the paper.

Specifically, we apply **hierarchical clustering** to group assets based on the similarity of their return profiles. Each cluster represents a group of highly correlated assets, and we solve the CCMV problem using the **cluster centroids** as representative assets.

This technique offers two key advantages:
- It **reduces the dimensionality** of the problem, making it more tractable.
- It **preserves key structure** in the data by exploiting the correlation between assets within each cluster.

By solving a reduced CCMV problem on the cluster level, we approximate the original high-dimensional problem while significantly improving computational efficiency.


### Implementation of Hierarchical Clustering 

We now implement the hierarchical clustering approach, following the pseudo-code provided in the paper. This algorithm clusters assets based on the similarity of their return profiles by analyzing the correlation structure of the return matrix.

The purpose of this step is to reduce the dimensionality of the portfolio optimization problem by grouping similar assets together. This allows us to simplify the later stages of solving the CCMV problem, while still preserving the core structure of the asset universe.

The clustering is performed using a bottom-up (agglomerative) approach, with distances computed between cluster centroids in the distance space. The final partition is determined by cutting the dendrogram at the optimal level, based on the largest gap in merge distances.

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [ ]:
def Hierarchical_Clustering(R: np.ndarray) -> np.ndarray:
	"""
	Perform hierarchical clustering on the given return matrix R.
	
	Parameters:
	R (np.ndarray): A return matrix.
	
	Returns:
	np.ndarray: A 1D array of clusters.
	"""

	# 1. Compute the correlation amtrix tau
	tau = np.corrcoef(R, rowvar=False)

	# 2. Construct the distance matrix D
	D = 1-tau

	# 3. Initialize clusters C
	C = [[i] for i in np.arange(R.shape[1])]

	def d(Ci, Cj):
		"""
		Compute the distance between two clusters Ci and Cj.
		
		Parameters:
		Ci (list): First cluster.
		Cj (list): Second cluster.
		
		Returns:
		float: The distance between the two clusters.
		"""
		cent_i = 1/ len(Ci) * np.sum(D[Ci, :], axis=0)
		cent_j = 1/ len(Cj) * np.sum(D[Cj, :], axis=0)
		dist = (len(Ci) * len(Cj)) / (len(Ci) + len(Cj)) * np.linalg.norm(cent_i - cent_j) ** 2
		return dist
	# 4. Repeat until only one cluster remains
	merge_distances = []
	merge_log = []
	while len(C) > 1:
		min_distance = float('inf')
		best_pair = (0, 0)
		for i in range(len(C)):
			for j in range(i+1, len(C)):
				distance = d(C[i], C[j])
				if distance < min_distance:
					min_distance = distance
					best_pair = (i, j)
		
		# Merge the best pair of clusters
		i, j = best_pair
		new_cluster = C[i] + C[j]
		merge_log.append((C[i], C[j], new_cluster, min_distance))
		del C[max(i, j)]  # Remove the larger index first to avoid index issues
		del C[min(i, j)]
		C.append(new_cluster)
		merge_distances.append(min_distance)

	# 5. Determine optimal number of clusters
	gap = []
	for k in range(1, len(merge_distances)):
		gap.append(merge_distances[k] - merge_distances[k-1])
	largest_gap = np.argmax(gap) + 1
	threshold = merge_distances[largest_gap]
	# 6. Assign clusters based on the threshold
	clusters = [{i} for i in range(R.shape[1])]

	for log in merge_log:
		if log[3] <= threshold:
			Ci, Cj = log[0], log[1]
			to_merge = []
			for cluster in clusters:
				if any(i in cluster for i in Ci) or any(i in cluster for i in Cj):
					to_merge.append(cluster)

			if len(to_merge) == 2:
				clusters.remove(to_merge[0])
				clusters.remove(to_merge[1])
				clusters.append(set(to_merge[0]).union(set(to_merge[1])))
	
	labels = np.empty(R.shape[1], dtype=int)
	for cluster_id, asset_set in enumerate(clusters):
		for asset in asset_set:
			labels[asset] = cluster_id
	return labels


	

					

	